<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/notebooks/week10/Week_10_One_step_retrosynthesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


🎯 학습 목표
- **역합성 분석(retrosynthesis analysis)**의 기본 개념과 template-based 접근 방식의 원리를 이해한다.

- **화학 반응 템플릿 (reaction templates)**의 정의와 역할, 그리고 reaction SMARTS 기반 표현 방식을 학습한다.

- RDKit과 RDChiral을 활용하여 템플릿을 분자에 적용하는 방법을 실습을 통해 익힌다.

- template extraction 및 template application의 전반적인 워크플로우를 살펴보고, reaction center 기반의 분해 전략을 체험한다.

- Template-matching 기반의 single-step retrosynthesis 모델이 작동하는 과정을 Python 코드를 통해 따라가며 이해한다.

- 생성된 전구체 후보를 buyable filter를 통해 필터링하고, 유효한 전구체 세트를 선택하는 과정을 실습한다.

- template-based 방식의 장점과 한계를 고찰하고, 이후 template-free 또는 hybrid 접근법으로의 확장 가능성에 대해 간단히 논의한다.

# Week 8: 역합성 분석 (Retrosynthetic analysis)

지금까지 이 강의에서는 머신러닝이 어떻게 분자의 특성을 분류하고 예측하는 데 도움을 줄 수 있는지, 원하는 특성을 가진 새로운 분자를 어떻게 생성할 수 있는지, 그리고 반응 혼합물의 반응성을 어떻게 예측할 수 있는지를 배웠습니다.

하지만 화학자의 작업에서 가장 핵심적인 질문은 바로 **어떻게 분자를 합성할 것인가?**입니다. 이 노트북에서는 바로 그 주제를 탐구합니다.

역합성 분석(retrosynthetic analysis)을 수행하기 위해, 우리는 목표(target) 분자에서 시작하여 어떤 변환이 원하는 생성물로 이어질 수 있을지를 추론합니다.

<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/retro_q.png" width="300"/>
</p>

## 우리는 어떻게 이런 결정을 내릴 수 있을까?

가장 단순한 접근 방식은, **가능한 모든 반응 변환들의 라이브러리**를 가지고 있고,  
그 중에서 **어떤 변환이 원하는 생성물을 만들 가능성이 높은지**를 판단하는 것입니다.

이러한 가능한 변환들의 목록에는 유기화학 수업에서 배운 여러 **명명 반응(name reactions)**이 포함됩니다:  
Grignard 반응, Diels-Alder 고리화 반응, Aldol 축합 반응 등.  
그 후 우리는 **화학 지식**을 바탕으로 어떤 반응이 생성물을 만들 수 있을지 판단하게 됩니다.

예를 들어, 아래 그림에서는 Grignard 반응이 타당하지만, Diels-Alder 반응은 그렇지 않습니다.


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/retro_1.png" width="500"/>
</p>

그렇다면, 이런 과정을 **모델이 어떻게 학습할 수 있을까요?**

**이 노트북에서는 반응 데이터셋으로부터 변환 라이브러리를 추출하고,  
이를 바탕으로 모델을 훈련시켜 단일 단계 역합성(one-step retrosynthesis)을 수행하는 방법을 배웁니다.**


# 0. 관련 패키지

## RDChiral

**RDChiral**은 RDKit의 반응 처리 기능을 감싸는(wrapper) 라이브러리로, 입체화학(stereochemistry) 처리를 개선한 도구입니다.  
이 패키지를 사용하면 반응 데이터셋으로부터 `reaction template`(반응 템플릿)을 추출할 수 있으며, 이는 **화학 변환 규칙(transformation rules)**을 표준화된 방식으로 표현한 것입니다.

또한 RDChiral은 이러한 `reaction template`을 목표 분자에 적용하여,  
주어진 변환 규칙 하에서 **어떤 반응물(reactants)이 해당 생성물(target molecule)을 만들 수 있는지**를 예측할 수 있도록 해줍니다.

더 자세한 정보는 [코드 저장소](https://github.com/connorcoley/rdchiral)와 [관련 논문](https://pubs.acs.org/doi/10.1021/acs.jcim.9b00286)을 참고하세요.


## 데이터 다운로드

이번 실습에서는 **USPTO 반응 데이터셋(USPTO reaction dataset)**에서 추출한  
50,000개의 반응으로 구성된 서브셋(subset)을 사용합니다.  

이 데이터셋은 역합성 예측 모델을 학습하고 평가하기 위해  
문헌에서도 자주 사용되는 표준 데이터셋입니다.


In [ ]:
! pip install reaction-utils
! pip install scikit-learn==1.2.2

**세션 재시작!**

In [ ]:
!mkdir -p data/
!curl -L "https://www.dropbox.com/scl/fi/6ixpkg8ke7nvsjms9dwti/uspto50k.zip?rlkey=i8vf64z307uh343n55ph7uzif&dl=1" -o data/uspto50k.zip
!unzip data/uspto50k.zip -d data/
!wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week10/utils.py

In [ ]:
from utils import load_data, visualize_chemical_reaction

train_df, val_df, test_df = load_data()

# 1. 반응 템플릿 (Reaction templates)

coupling reaction을 예시로 살펴보겠습니다.

In [ ]:
rxn_example = train_df.iloc[5,0]

visualize_chemical_reaction(rxn_example)

### 반응 템플릿을 추출하려면 `utils.py`에 있는 `extract_template` 함수를 사용하세요.

**반응 템플릿(reaction template)**은 특정 유형의 일반적인 화학 변환을 나타냅니다.  
이 템플릿은 변환 과정에서 **어떤 결합이 생성되고 끊어지는지**,  
그리고 그 결합이 위치한 **화학적 환경(chemical environment)**을 함께 기술합니다.

In [ ]:
from utils import extract_template

tplt_example = extract_template(rxn_example)

# A reaction template looks like this
print(tplt_example)

### 이제 이 반응 템플릿을 사용할 수 있습니다. `utils.py`에 있는 `apply_template` 함수를 사용하세요.

같은 product에 이 템플릿을 적용하면, 앞서 본 것과 동일한 반응물(reactants)을 얻을 수 있어야 합니다.


In [ ]:
from utils import apply_template, visualize_mols

# Apply the extracted template to the product above.
prod_1 = rxn_example.split('>>')[1]
pred_reactants = apply_template(tplt_example, prod_1)

# This is the result of applying the template.
display(pred_reactants[0])
display(pred_reactants[1])

### 반응 템플릿이 일반적인 변환을 어떻게 표현하는지 확인하기 위해, 이제 이 템플릿을 다른 생성물에 적용해 보겠습니다.


In [ ]:
new_prod = 'CC1=CC(C2=CN=CN2C)=CC=C1'
# Let's try to break down this product, using the newly extracted reaction template.
visualize_mols(new_prod)

In [ ]:
new_reactants = apply_template(tplt_example, new_prod)
display(new_reactants[0])
display(new_reactants[1])

## 우리는 Bromo Suzuki coupling을 성공적으로 템플릿으로 표현했습니다!

보시다시피, 이 반응 템플릿은 이제 어떤 생성물에도 적용할 수 있으며,  
그 결과로 해당 생성물을 만들기 위해 필요한 반응물들이 출력됩니다.  
이러한 반응 템플릿에 대해 더 알고 싶다면 [여기](https://chimia.ch/chimia/article/view/2022_294/5301)를 참고하세요.

**그렇다면, 이 반응 유형에 맞지 않는 생성물에 템플릿을 적용하면 어떤 일이 벌어질까요?**


# 2. 반응 템플릿 라이브러리 구축

이제 학습 데이터에서 가능한 모든 반응 템플릿을 추출해보겠습니다.  
이렇게 생성된 템플릿 라이브러리를 이용하면,  
주어진 생성물(product)에 가장 적합한 템플릿을 선택하는 모델을 학습할 수 있습니다!

### 이 작업은 약 10분 정도 소요됩니다.


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm  # 진행 상황 표시를 위한 tqdm 모듈
tqdm.pandas()

# 전체 데이터셋을 사용할지 여부를 결정 (True이면 샘플만 사용)
SAMPLE = True

# 샘플 모드일 경우, 데이터의 10%만 사용
if SAMPLE:
    train_df = train_df.sample(frac=0.1)
    val_df = val_df.sample(frac=0.1)

# 학습 및 검증 데이터에서 반응 템플릿 추출
# 'reactants>reagents>production' 컬럼에 대해 extract_template 함수를 적용
train_df['tmplt'] = train_df['reactants>reagents>production'].progress_apply(extract_template)
val_df['tmplt'] = val_df['reactants>reagents>production'].progress_apply(extract_template)

# 학습/검증 데이터에서 추출된 템플릿을 합치고, 등장 빈도 기준으로 정렬하여 고유한 템플릿 목록 생성
template_library = (pd.concat([train_df['tmplt'], val_df['tmplt']])
                    .value_counts()     # 템플릿 빈도 계산
                    .index              # 템플릿 문자열만 추출
                    .values             # numpy array로 변환
                   )

# 전체 템플릿 수 출력
print(f'We have {len(template_library)} different templates!')


In [ ]:
# 이제 각 반응 템플릿에 고유한 클래스 번호를 부여합니다.
import numpy as np

# 학습 데이터에 대해:
# 각 템플릿 문자열 x에 대해 template_library에서의 인덱스를 찾아 rxn_class로 저장
train_df['rxn_class'] = train_df['tmplt'].progress_apply(lambda x: np.where(template_library == x)[0][0])

# 검증 데이터에 대해서도 동일하게 처리
val_df['rxn_class'] = val_df['tmplt'].progress_apply(lambda x: np.where(template_library == x)[0][0])


# 3. 역합성 예측을 위한 머신러닝 모델 학습

우리는 지금 11,000개가 넘는 템플릿을 가지고 있고, 각 product에 대해 어떤 템플릿을 적용해야 할지를 결정해야 합니다.  
이런 문제에 가장 적합한 모델은 **분류(classification) 모델**이며,  
여러분은 이 강의 초반에 이미 몇 가지 분류 모델을 배운 바 있습니다!

이전에는 분자가 독성인지 여부(2개의 클래스)를 예측했다면, 이제는 **어떤 템플릿을 써야 하는지**를 예측하는 문제로 바뀌었고, 클래스 수는 11,000개 이상입니다.

**기억하시겠지만, 분자를 모델이 읽기 쉬운 형태로 표현해야 합니다.**  
여기서는 그 표현 방식으로 **Morgan Fingerprint**를 사용하겠습니다.

`utils.py`에서 `mol_smi_to_count_fp` 함수를 import 하세요.


In [ ]:
from utils import mol_smi_to_count_fp  # SMILES 문자열을 Morgan fingerprint로 변환하는 함수

# 학습 데이터에 대한 fingerprint 생성
print('학습 데이터의 fingerprint 생성 중')
train_fps = train_df['reactants>reagents>production'].progress_apply(lambda x: x.split('>>')[1]).progress_apply(mol_smi_to_count_fp)
                                                        # 생성물(SMILES) 추출                     # fingerprint 변환 적용

# 검증 데이터에 대한 fingerprint 생성
print('검증 데이터의 fingerprint 생성 중')
val_fps = val_df['reactants>reagents>production'].progress_apply(lambda x: x.split('>>')[1]).progress_apply(mol_smi_to_count_fp)

# 테스트 데이터에 대한 fingerprint 생성
print('테스트 데이터의 fingerprint 생성 중')
test_fps = test_df['reactants>reagents>production'].progress_apply(lambda x: x.split('>>')[1]).progress_apply(mol_smi_to_count_fp)

# 이제 머신러닝을 활용해 역합성 문제에 도전해볼 수 있습니다!
### 이 문제는 클래스 수가 매우 많고, 각 클래스(템플릿)를 명확히 정의할 방법이 없기 때문에, 분류(classification) 모델이 적합한 접근법입니다.


In [ ]:
train_fps_dense = np.asarray(np.concatenate([v.todense() for v in train_fps.values]))
val_fps_dense = np.asarray(np.concatenate([v.todense() for v in val_fps.values]))

In [ ]:
# Let's define a very simple random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    random_state=0,
    n_jobs=20,
    verbose=1
)

rf_clf.fit(train_fps_dense, train_df['rxn_class'])

### 모델 테스트

모델을 평가하는 방법은 두 가지가 있습니다:
- 템플릿 예측 정확도 (Accuracy of template prediction)
- 전구체 예측 정확도 (Accuracy of precursor prediction)

이 둘은 실제로 **다를 수 있습니다!**  
**왜 그런 걸까요?**


In [ ]:
# 검증 데이터에 대해 템플릿 분류 정확도를 확인해봅시다

# 검증 데이터의 fingerprint에 대해 예측 수행
val_preds = rf_clf.predict(val_fps_dense)

# 예측 결과와 실제 클래스(rxn_class)가 일치하는 비율 계산
val_acc = (val_preds == val_df['rxn_class']).mean()

# 템플릿 예측 정확도 출력
print(f'템플릿 예측에 대한 검증 정확도: {val_acc*100:.3f}%')


In [ ]:
from rdkit import Chem

# 랜덤 포레스트 분류기로 예측한 템플릿을 DataFrame에 저장
val_df['pred_tmplt_rfclf'] = template_library[val_preds]

# 예측한 템플릿을 실제 생성물(product)에 적용하여 전구체(reactants) 예측

def get_precs_template(tmplt, prod):
    # apply_template 함수로 템플릿 적용
    pred = apply_template(
        tmplt,
        prod
    )
    if pred:
        # 생성된 전구체들을 SMILES로 변환하고 '.'로 연결
        prods = [Chem.MolToSmiles(mol) for mol in pred]
        final = ".".join(prods)
        return final
    else:
        # 템플릿 적용이 실패하면 invalid 처리
        return 'Invalid template'

# 각 행에 대해 예측한 템플릿을 product에 적용하여 전구체 예측 결과 저장
val_df['pred_prec'] = val_df.apply(
    lambda x: get_precs_template(
        x['pred_tmplt_rfclf'],
        x['reactants>reagents>production'].split('>>')[1]  # 생성물 SMILES 추출
    ),
    axis=1
)

from utils import canonicalize_smiles

# 실제 반응물 SMILES를 canonicalize 처리하여 비교 기준 생성
val_df['prec_true'] = val_df['reactants>reagents>production'].apply(lambda x: canonicalize_smiles(x.split('>>')[0]))
val_df['pred_prec'] = val_df['pred_prec'].apply(canonicalize_smiles)

# 예측된 전구체가 실제 전구체와 일치하는 비율 계산
prec_acc = (val_df['pred_prec'] == val_df['prec_true']).mean()

# 전구체 예측 정확도 출력
print(f'전구체 예측 정확도: {prec_acc*100:.3f}%')


In [ ]:
# 정확도가 낮게 나왔는데, 그 이유는 뭘까?

# 템플릿 적용이 실패하지 않은 (빈 문자열이 아닌) 경우의 개수를 출력
print(f'템플릿이 유효했던 경우: {(val_df["pred_prec"]!="").sum()}개 / 전체 {val_df.shape[0]}개 중')

# 유효한 템플릿만 필터링하여 정확도 다시 계산
val_df_valid_preds = val_df[val_df['pred_prec'] != '']

# 유효한 템플릿에 대해서만 전구체 예측 정확도 계산
prec_acc_valid_preds = (val_df_valid_preds['pred_prec'] == val_df_valid_preds['prec_true']).mean()

# 유효한 템플릿에 한해서 전구체 예측 정확도 출력
print(f'전구체 예측 정확도 (유효한 템플릿만 고려 시): {prec_acc_valid_preds*100:.3f}%')


# 이러한 문제들을 어떻게 해결할 수 있을까?

- 현재 모델은 하나의 클래스만 예측합니다. 이를 개선하는 한 가지 방법은 **여러 개의 클래스를 예측한 후**,  
  **유효한 템플릿이 나올 때까지 순차적으로 적용**해보는 것입니다.

- 또는 **더 나은 모델을 학습**시키는 방법도 있습니다.

## 이 데이터셋에서 최고 성능을 보이는 모델들은 정확도 55% 이상을 달성합니다  
([리더보드 보기](https://paperswithcode.com/sota/single-step-retrosynthesis-on-uspto-50k)).  
이 단순한 모델은 개선 여지가 매우 많습니다.  
강의에서 배운 다양한 기법들을 활용해 직접 개선해보세요!
---


# 4. 이제 우리가 만든 모델을 활용해, 여러분이 관심 있는 분자를 합성해봅시다!


In [ ]:
# 생성물(product)이 주어졌을 때, 예측된 전구체(reactants)를 출력하는 함수
def predict_precursors(product):

    # 제품 분자의 fingerprint 생성 및 dense format 변환
    fp = mol_smi_to_count_fp(product).todense()

    # 랜덤 포레스트 분류기로 템플릿별 확률(logits) 예측
    logits = rf_clf.predict_proba(np.asarray(fp))

    # 확률이 높은 템플릿 순으로 정렬
    sorted_templts = np.argsort(logits)[0][::-1]

    # 각 템플릿을 높은 순서대로 하나씩 적용해보기
    for i, t in enumerate(sorted_templts):
        template = template_library[t]
        try:
            # 템플릿을 적용하여 전구체 예측
            prec = get_precs_template(template, product)
        except:
            # 예외 발생 시 다음 템플릿으로
            continue

        # 유효한 템플릿이 적용된 경우
        if prec != 'Invalid template':
            print(f'{i}번째 템플릿이 사용되었습니다.\n\n')
            print('이 분자를 합성하려면:')
            visualize_mols(product)  # 생성물 시각화
            print('랜덤 포레스트 모델은 다음 반응물들을 예측합니다:')
            visualize_mols(prec)     # 예측된 반응물 시각화
            return prec


In [ ]:

# 관심 있는 분자에 대해 전구체 예측 실행
reacts = predict_precursors('CC1=CC(C2=CN=CN2C)=CC=C1')


### 추가 학습 자료

데이터 기반 역합성(data-driven retrosynthesis)에 대해 더 배우고 싶다면, 아래의 자료들을 추천합니다:

- [템플릿 기반 단일 단계 역합성](https://chemistry-europe.onlinelibrary.wiley.com/doi/full/10.1002/chem.201605499)  
- [다단계 역합성(Multi-step retrosynthesis)](https://www.nature.com/articles/nature25978)
